# デザイン実験

## データの読み込み

In [155]:
from pathlib import Path

import numpy as np
import pandas as pd


result_path = Path('result')
design_result_path = result_path / 'design_experiment'
dryer_1_folder = design_result_path / 'condition_1' / 'dryer'
dryer_2_folder = design_result_path / 'condition_2' / 'dryer'
fridge_1_folder = design_result_path / 'condition_1' / 'fridge'
fridge_2_folder = design_result_path / 'condition_2' / 'fridge'

dryer_1_paths = list(dryer_1_folder.glob('*.csv'))
dryer_2_paths = list(dryer_2_folder.glob('*.csv'))
fridge_1_paths = list(fridge_1_folder.glob('*.csv'))
fridge_2_paths = list(fridge_2_folder.glob('*.csv'))

In [156]:
path_lists = [list(dryer_1_paths), list(dryer_2_paths), list(fridge_1_paths), list(fridge_2_paths)]

In [157]:
subject_dict = {'okamoto':1,
                'matsuyama':2,
                'nakano':3,
                'hayashi':4,
                'shimizu':5,
                'uchida':6,
                'kokado':7,
                'kitagawa':8,
                }

In [158]:
def get_subject_name(filename, subject_dict):
    for key in subject_dict.keys():
        if key in filename:
            return subject_dict[key]
            

def get_desing_name(filename):
    if 'dryer' in filename:
        return 'dryer'
    elif 'fridge' in filename:
        return 'fridge'

def get_system_name(filename):
    if 'condition_1' in filename:
        return 'B'
    elif 'condition_2' in filename:
        return 'A'

In [159]:
# 先にカラムのみのdf作る
df = pd.DataFrame(None, columns=['subject_ID', 'design_target', 'system', 'count', 'function', 'id', 'label', 'is_selected', 'not_is_selected'])
for path_list in path_lists:
    for path in path_list:
        # それぞれのdf作る
        new_df = pd.read_csv(path)
        new_df.insert(0, 'subject_ID', get_subject_name(str(path), subject_dict))
        new_df.insert(1, 'design_target', get_desing_name(str(path)))
        new_df.insert(2, 'system', get_system_name(str(path)))
        new_df.insert(8, 'not_is_selected', ~new_df['is_selected'])
        df = pd.concat([df, new_df])
        # dfに連結
df = df.reset_index(drop=True)
df.loc[df['function'] == 0, 'new_function'] = 0
df.loc[(df['function'] == 1) | (df['function'] == 2 ) | (df['function'] == 3 ), 'new_function'] = 1
df.loc[df['function'] == 4, 'new_function'] = 2
df.loc[df['function'] == 5, 'new_function'] = 3
df.loc[df['function'] == 10, 'new_function'] = 10
df.loc[df['function'] == 11, 'new_function'] = 11

df.to_csv('単語履歴.csv')

In [160]:
df

,subject_ID,design_target,system,count,function,id,label,is_selected,not_is_selected,new_function
0,3,dryer,B,1,0,0,dryer,False,True,0.0
1,3,dryer,B,1,1,1,appliance,False,True,1.0
2,3,dryer,B,1,2,2,clothes_dryer,True,False,1.0
3,3,dryer,B,1,3,3,clothes_dryer,False,True,1.0
4,3,dryer,B,1,4,4,washing_machine,False,True,2.0
...,...,...,...,...,...,...,...,...,...,...
4529,3,fridge,A,4,11,39,laa,False,True,11.0
4530,3,fridge,A,4,11,40,restuarant,False,True,11.0
4531,3,fridge,A,4,11,41,pizza_parlor,False,True,11.0
4532,3,fridge,A,4,11,42,Grille,False,True,11.0


デザイン案と機能の対応

In [170]:
df_temp = df[['subject_ID', 'design_target', 'system', 'new_function', 'label']]
df_temp

,subject_ID,design_target,system,new_function,label
0,3,dryer,B,0.0,dryer
1,3,dryer,B,1.0,appliance
2,3,dryer,B,1.0,clothes_dryer
3,3,dryer,B,1.0,clothes_dryer
4,3,dryer,B,2.0,washing_machine
...,...,...,...,...,...
4529,3,fridge,A,11.0,laa
4530,3,fridge,A,11.0,restuarant
4531,3,fridge,A,11.0,pizza_parlor
4532,3,fridge,A,11.0,Grille


In [175]:
def get_function(subject_ID, design_target, system, label):
    # print(subject_ID, design_target, system, label)
    num_function = df_temp[(df_temp['subject_ID'] == subject_ID) & (df_temp['design_target'] == design_target) & (df_temp['system'] == system) & (df_temp['label'] == label)]['new_function'].values
    
    num_function = set(num_function)
    nums = ''
    for i in num_function:
        nums += str(int(i)) + ','
    return nums

In [176]:
get_function(3, 'dryer', 'B', 'dryer')

'0,'

In [177]:
design_plans = pd.read_csv('デザイン案.csv')
design_plans.columns = ['subject_ID', 'design_target', 'system', 'design_plan', 'word']
design_plans.loc[design_plans['design_target'] == 'ドライヤー', 'design_target'] = 'dryer'
design_plans.loc[design_plans['design_target'] == '冷蔵庫', 'design_target'] = 'fridge'
design_plans['num_function'] = None
for i in range(len(design_plans)):
    subject_ID = int(design_plans.at[i, 'subject_ID'])
    design_target = design_plans.at[i, 'design_target']
    system = design_plans.at[i, 'system']
    label = design_plans.at[i, 'word']
    design_plans.at[i, 'num_function'] = get_function(subject_ID, design_target, system, label)
    

In [179]:
design_plans.to_csv('デザイン案と機能.csv')
design_plans

,subject_ID,design_target,system,design_plan,word,num_function
0,1,dryer,A,可愛いドライヤー,lobeeliees,
1,1,dryer,A,髪をくねらせるドライヤー,curly_hair,"10,"
2,1,dryer,A,寝癖に水をかけられるドライヤー,wet_washer,
3,1,dryer,A,濡らす機能付きドライヤー,wet_washer,
4,1,dryer,A,無音のドライヤー,chill,"0,10,"
...,...,...,...,...,...,...
162,８,fridge,B,人も入って栄養や地元野菜について考えられる冷蔵庫,nutriment,"1,"
163,８,fridge,B,ルイベや可冷却水を提供できる冷蔵庫,deep-freeze,"0,1,"
164,８,fridge,B,鉄板焼きの反対、鉄板冷やし的な冷蔵庫,shape,"0,"
165,８,fridge,B,皆で冷菜？を夏に食べれる曲線的な冷却棚を持った冷蔵庫,spiral,"0,2,3,"


In [73]:
df_selected_per_function = df.groupby(['design_target', 'system', 'new_function']).sum().drop(['function', 'subject_ID', 'count', 'id', 'label'], axis=1)
df_selected_per_function['ratio'] = df_selected_per_function['is_selected']/(df_selected_per_function['is_selected'] + df_selected_per_function['not_is_selected'])
df_selected_per_function['ratio'].to_csv('機能ごとの採択率.csv')

In [62]:
df.groupby(['design_target', 'system', 'function']).sum().drop(['subject_ID', 'count', 'id', 'label'], axis=1)

KeyError: "['subject_ID' 'count' 'id' 'label'] not found in axis"

In [78]:
df.groupby(['design_target', 'system']).count()

subject_ID  count  function    id  label  is_selected  \
design_target system                                                          
dryer         A             1262   1262      1262  1262   1262         1262   
              B             1016   1016      1016  1016   1016         1016   
fridge        A             1322   1322      1322  1322   1322         1322   
              B              934    934       934   934    934          934   

                      not_is_selected  
design_target system                   
dryer         A                  1262  
              B                  1016  
fridge        A                  1322  
              B                   934

In [86]:
df_result = df.groupby(['design_target', 'system', 'subject_ID']).sum()
df_result['ratio'] = df_result['is_selected']/(df_result['is_selected'] + df_result['not_is_selected'])
df_result['ratio']

design_target  system  subject_ID
dryer          A       1             0.048930
                       4             0.060526
                       5             0.041009
                       8             0.088235
               B       2             0.031949
                       3             0.128571
                       6             0.073314
                       7             0.148649
fridge         A       2             0.021277
                       3             0.035714
                       6             0.035565
                       7             0.101604
               B       1             0.046784
                       4             0.089844
                       5             0.042735
                       8             0.274510
Name: ratio, dtype: float64

# 単語実験

In [215]:
import pandas as pd
import numpy as np
from pathlib import Path

result_path = Path('result')
word_result_path = result_path / 'word_experiment'
rawdata = word_result_path / 'rawdata.csv'
layout = word_result_path / 'layout.csv'
labeldata = word_result_path / 'labeldata.csv'

df_rawdata = pd.read_csv(rawdata, encoding='shift_jis')
df_layout = pd.read_csv(layout, encoding='shift_jis')
df_labeldata = pd.read_csv(labeldata, encoding='shift_jis')



df_rawdata = df_rawdata.drop(['ANSWERDATE', 'F1_SEX', 'F2_AGEID', 'F3_PREFECTURE', 'CELL', 'CELLNAME', 'SAMPLEID'], axis=1)
df_rawdata = df_rawdata.drop(['Q1S1', 'Q1S2', 'Q1S3', 'Q1S4', 'Q1S5', 'Q2S1', 'Q2S2', 'Q2S3', 'Q2S4', 'Q2S5'], axis=1).transpose().reset_index()
df_rawdata = df_rawdata.add_prefix('id_')
df_rawdata.insert(1, 'question_type', None)
df_rawdata.insert(1, 'method', None)
df_rawdata.insert(1, 'word', None)

df_q_and_output_word = df_layout[['アイテム名', '質問文B']]
df_q_and_output_word = df_q_and_output_word[7:].reset_index(drop=True)
# df_rawdata.insert(4, 'output_word', None)
df_rawdata.insert(4, 'means_per_question', 0)
df_rawdata.insert(4, 'std_per_question', 0)

df_rawdata['means_per_question'] = df_rawdata.drop(['id_index','word','method','question_type','means_per_question', 'std_per_question'],axis=1).mean(axis=1)
df_rawdata['std_per_question'] = df_rawdata.drop(['id_index','word','method','question_type','means_per_question', 'std_per_question'],axis=1).std(axis=1)

df_rawdata.loc[df_rawdata['id_index'].str.contains('Q3|Q5|Q7'), 'question_type'] = 'relation'
df_rawdata.loc[df_rawdata['id_index'].str.contains('Q4|Q6|Q8'), 'question_type'] = 'design'

df_rawdata.loc[df_rawdata['id_index'].str.contains('Q3|Q4'), 'word'] = 'suit'
df_rawdata.loc[df_rawdata['id_index'].str.contains('Q5|Q6'), 'word'] = 'chair'
df_rawdata.loc[df_rawdata['id_index'].str.contains('Q7|Q8'), 'word'] = 'table'

df_rawdata.loc[df_rawdata['id_index'].str.contains('S1|S2|S3|S4|S5|S6|S7|S8|S9|S10'), 'method'] = 'Word2vec'
df_rawdata.loc[df_rawdata['id_index'].str.contains('S11|S12|S13|S14|S15|S16|S17|S18|S19|S20'), 'method'] = 'AutoExtend'
df_rawdata.loc[df_rawdata['id_index'].str.contains('S21|S22|S23|S24|S25|S26'), 'method'] = 'hyper_hypo'
df_rawdata.loc[df_rawdata['id_index'].str.contains('S27|S28|S29|S30|S31'), 'method'] = 'analogy_sim'
df_rawdata.loc[df_rawdata['id_index'].str.contains('S32|S33|S34|S35|S36'), 'method'] = 'analogy_hyper_hypo'
# df_rawdata = df_rawdata[['id_index', 'word', 'method', 'question_type', 'means_per_question', 'std_per_question']]

df_q_and_output_word.columns = ['id_index', 'output_word_temp']
df_rawdata = pd.merge(df_rawdata, df_q_and_output_word, on='id_index')
df_rawdata.insert(4, 'output_word', df_rawdata['output_word_temp'].to_list())
df_rawdata = df_rawdata.drop('output_word_temp', axis=1)

# id_35を削除(同じ質問で，3以上離れている)
df_rawdata = df_rawdata.drop('id_45', axis=1)
# id_12を削除(ほぼ3)
df_rawdata = df_rawdata.drop('id_12', axis=1)

df_relation_per_question = df_rawdata[df_rawdata['question_type'] == 'relation'].drop(['question_type', 'id_index'], axis=True).reset_index(drop=True)
df_relation_per_question = df_relation_per_question[['word', 'method', 'output_word', 'std_per_question', 'means_per_question']]
df_design_per_question = df_rawdata[df_rawdata['question_type'] == 'design'].drop(['question_type', 'id_index'], axis=True).reset_index(drop=True)
df_design_per_question = df_design_per_question[['word', 'method', 'output_word', 'std_per_question', 'means_per_question']]


df_relation = df_rawdata[df_rawdata['question_type'] == 'relation'].drop(['question_type', 'id_index', 'output_word'], axis=True).reset_index(drop=True)
df_design = df_rawdata[df_rawdata['question_type'] == 'design'].drop(['question_type', 'id_index', 'output_word'], axis=True).reset_index(drop=True)

質問ごとの点数

関係

In [216]:
df_relation_per_question.to_csv('質問ごとの点数(関係性).csv')
df_design_per_question.to_csv('質問ごとの点数(デザインに有効).csv')

In [217]:
def get_points_list(df: pd.DataFrame, word, method):
    df = df[(df['method'] == method) & (df['word'] == word)].reset_index(drop=True)
    points = []
    for i in range(len(df)):
        points = points + df.iloc[i][2:].values.tolist()
    return points

def convert_dataframe(df, words, methods, question_type):
    new_df = pd.DataFrame(None, columns=['word', methods[0], methods[1]])
    for word in words:
        points_method1 = get_points_list(df, word, methods[0])
        points_method2 = get_points_list(df, word, methods[1])
        df_temp = pd.DataFrame({'word':word,
                        methods[0]:points_method1,
                        methods[1]:points_method2
                        })
        new_df = pd.concat([new_df, df_temp])
    new_df = new_df.reset_index(drop=True)    
    new_df.to_csv(f'result/csv/statistics/{question_type}_{methods[0]}_{methods[1]}.csv', index=False)

## 関連

### 類似度

In [243]:
df_relation_simi = df_relation[(df_relation['method'] == 'Word2vec') | (df_relation['method'] == 'AutoExtend')]

In [244]:
df_relation_simi.groupby(['word', 'method']).describe()

std_per_question                                          \
                            count      mean       std       min       25%   
word  method                                                                
chair AutoExtend             10.0  0.923535  0.200111  0.741733  0.835049   
      Word2vec               10.0  1.073368  0.221215  0.739460  0.878723   
suit  AutoExtend             10.0  0.996430  0.158044  0.813191  0.893645   
      Word2vec               10.0  0.873539  0.106786  0.765414  0.787207   
table AutoExtend             10.0  0.936346  0.140902  0.764093  0.818964   
      Word2vec               10.0  1.004678  0.071189  0.899682  0.965341   

                                               means_per_question            \
                       50%       75%       max              count      mean   
word  method                                                                  
chair AutoExtend  0.875210  0.890055  1.445299               10.0  4.261818   
      Word2vec    1.192611  1.226735  1.242350               10.0  2.923636   
suit  AutoExtend  0.956100  1.070190  1.263313               10.0  4.074545   
      Word2vec    0.856830  0.929667  1.084634               10.0  2.218182   
table AutoExtend  0.897807  1.053663  1.161098               10.0  3.714545   
      Word2vec    1.012200  1.041309  1.125762               10.0  3.134545   

                  ... id_53      id_54                                       \
                  ...   75%  max count mean       std  min   25%  50%   75%   
word  method      ...                                                         
chair AutoExtend  ...  5.00  5.0  10.0  4.5  1.080123  2.0  5.00  5.0  5.00   
      Word2vec    ...  4.50  5.0  10.0  2.2  1.932184  1.0  1.00  1.0  4.00   
suit  AutoExtend  ...  5.00  5.0  10.0  4.0  1.247219  2.0  3.25  4.5  5.00   
      Word2vec    ...  3.75  5.0  10.0  2.0  1.699673  1.0  1.00  1.0  2.50   
table AutoExtend  ...  5.00  5.0  10.0  3.7  0.674949  3.0  3.00  4.0  4.00   
      Word2vec    ...  5.00  5.0  10.0  3.2  0.918937  2.0  3.00  3.0  3.75   

                       
                  max  
word  method           
chair AutoExtend  5.0  
      Word2vec    5.0  
suit  AutoExtend  5.0  
      Word2vec    5.0  
table AutoExtend  5.0  
      Word2vec    5.0  

[6 rows x 440 columns]

### 上位下位

In [246]:
df_relation_hypo = df_relation[(df_relation['method'] == 'AutoExtend') | (df_relation['method'] == 'hyper_hypo')]
# df_relation_hypo = df_relation_hypo.drop([6, 7, 8, 9, 42, 43, 44, 45, 78, 79, 80, 81]) # 7〜10位を削除
df_relation_hypo = df_relation_hypo.drop([15, 16, 17, 18, 51, 52, 53, 54, 87, 88, 89, 90]).drop(['std_per_question', 'means_per_question'], axis=1) # 6〜10を削除
df_relation_hypo

,word,method,id_0,id_1,id_2,id_3,id_4,id_5,id_6,id_7,...,id_44,id_46,id_47,id_48,id_49,id_50,id_51,id_52,id_53,id_54
10,suit,AutoExtend,2,3,5,4,4,4,4,3,...,3,5,5,5,1,3,1,5,5,5
11,suit,AutoExtend,2,2,5,5,4,5,3,5,...,5,5,5,5,3,3,5,5,5,4
12,suit,AutoExtend,2,3,1,3,2,2,1,1,...,2,1,2,1,2,3,1,1,3,3
13,suit,AutoExtend,3,3,5,5,4,5,4,3,...,4,5,5,5,3,3,5,5,5,2
14,suit,AutoExtend,2,4,5,5,4,5,4,4,...,4,5,4,5,2,3,5,5,5,5
19,suit,AutoExtend,2,3,5,5,4,5,4,4,...,5,5,4,5,1,3,5,5,5,4
20,suit,hyper_hypo,2,3,5,5,4,5,4,5,...,5,5,5,5,3,3,5,5,5,4
21,suit,hyper_hypo,2,3,5,5,4,5,4,5,...,5,5,4,5,2,3,5,5,5,4
22,suit,hyper_hypo,2,3,5,5,5,5,5,5,...,5,5,5,5,4,2,5,5,5,5
23,suit,hyper_hypo,3,3,5,4,4,5,4,4,...,5,5,4,5,4,2,5,5,5,4


### 類推

In [247]:
df_relation_analogy = df_relation[(df_relation['method'] == 'AutoExtend') | (df_relation['method'] == 'analogy_hyper_hypo')]
df_relation_analogy = df_relation_analogy.drop([15, 16, 17, 18, 19, 51, 52, 53, 54, 55, 87, 88, 89, 90, 91])
df_relation_analogy

,word,method,std_per_question,means_per_question,id_0,id_1,id_2,id_3,id_4,id_5,...,id_44,id_46,id_47,id_48,id_49,id_50,id_51,id_52,id_53,id_54
10,suit,AutoExtend,1.244517,3.545455,2,3,5,4,4,4,...,3,5,5,5,1,3,1,5,5,5
11,suit,AutoExtend,0.952403,4.381818,2,2,5,5,4,5,...,5,5,5,5,3,3,5,5,5,4
12,suit,AutoExtend,1.263313,2.181818,2,3,1,3,2,2,...,2,1,2,1,2,3,1,1,3,3
13,suit,AutoExtend,1.100046,4.109091,3,3,5,5,4,5,...,4,5,5,5,3,3,5,5,5,2
14,suit,AutoExtend,0.832019,4.418182,2,4,5,5,4,5,...,4,5,4,5,2,3,5,5,5,5
31,suit,analogy_hyper_hypo,0.833636,4.436364,3,3,5,5,4,5,...,5,5,5,5,2,3,5,5,5,4
32,suit,analogy_hyper_hypo,0.840875,1.818182,2,3,2,3,3,2,...,2,1,2,2,1,3,1,1,3,3
33,suit,analogy_hyper_hypo,0.982336,4.327273,2,3,5,5,4,5,...,5,5,5,5,2,3,5,5,5,4
34,suit,analogy_hyper_hypo,1.184069,2.472727,2,3,2,4,4,3,...,1,1,2,1,3,3,1,1,5,3
35,suit,analogy_hyper_hypo,0.999326,4.236364,2,3,5,5,4,5,...,5,5,4,5,2,3,5,5,5,4


In [248]:
words = ['suit', 'chair', 'table']

methods = ['Word2vec', 'AutoExtend']
convert_dataframe(df_relation_simi, words, methods, question_type='relation')

methods = ['AutoExtend', 'hyper_hypo']
convert_dataframe(df_relation_hypo, words, methods, question_type='relation')

methods = ['AutoExtend', 'analogy_hyper_hypo']
convert_dataframe(df_relation_analogy, words, methods, question_type='relation')


## デザインに有効か

### 類似

In [238]:
df_design_simi = df_design[(df_design['method'] == 'Word2vec') | (df_design['method'] == 'AutoExtend')]

### 上位下位

In [239]:
df_design_hypo = df_design[(df_design['method'] == 'AutoExtend') | (df_design['method'] == 'hyper_hypo')]
df_design_hypo = df_design_hypo.drop([15, 16, 17, 18, 51, 52, 53, 54, 87, 88, 89, 90]).drop(['std_per_question', 'means_per_question'], axis=1) # 6〜10を削除


### 類推

In [240]:
df_design_analogy = df_design[(df_design['method'] == 'AutoExtend') | (df_design['method'] == 'analogy_hyper_hypo')]
df_design_analogy = df_design_analogy.drop([15, 16, 17, 18, 19, 51, 52, 53, 54, 55, 87, 88, 89, 90, 91]) 

In [242]:
words = ['suit', 'chair', 'table']

methods = ['Word2vec', 'AutoExtend']
convert_dataframe(df_design_simi, words, methods, question_type='design')

methods = ['AutoExtend', 'hyper_hypo']
convert_dataframe(df_design_hypo, words, methods, question_type='design')

methods = ['AutoExtend', 'analogy_hyper_hypo']
convert_dataframe(df_design_analogy, words, methods, question_type='design')


## ?

In [132]:
df_rawdata.to_csv('word_experiment')

In [133]:
df_rawdata.groupby(['question_type', 'word', 'method']).mean()

id_0      id_1      id_2  \
question_type word  method                                             
design        chair AutoExtend          3.500000  2.700000  4.300000   
                    Word2vec            3.500000  3.500000  2.200000   
                    analogy_hyper_hypo  3.600000  3.200000  4.400000   
                    analogy_sim         3.800000  3.200000  2.600000   
                    hyper_hypo          3.833333  3.000000  4.333333   
              suit  AutoExtend          3.300000  3.300000  4.200000   
                    Word2vec            2.700000  3.000000  2.100000   
                    analogy_hyper_hypo  3.200000  3.800000  3.600000   
                    analogy_sim         2.200000  3.400000  1.800000   
                    hyper_hypo          3.333333  3.666667  4.666667   
              table AutoExtend          3.300000  2.900000  3.900000   
                    Word2vec            2.900000  3.000000  3.000000   
                    analogy_hyper_hypo  3.000000  3.400000  3.800000   
                    analogy_sim         3.400000  3.000000  3.400000   
                    hyper_hypo          3.833333  3.333333  4.166667   
relation      chair AutoExtend          4.000000  3.300000  4.600000   
                    Word2vec            3.700000  3.100000  2.800000   
                    analogy_hyper_hypo  3.600000  3.000000  4.400000   
                    analogy_sim         3.600000  3.400000  3.000000   
                    hyper_hypo          4.333333  3.500000  4.333333   
              suit  AutoExtend          2.100000  3.000000  4.600000   
                    Word2vec            2.300000  3.300000  2.100000   
                    analogy_hyper_hypo  2.200000  3.000000  3.800000   
                    analogy_sim         2.200000  3.000000  1.200000   
                    hyper_hypo          2.333333  3.000000  4.833333   
              table AutoExtend          4.300000  3.400000  4.300000   
                    Word2vec            3.400000  3.300000  3.200000   
                    analogy_hyper_hypo  4.200000  3.400000  4.000000   
                    analogy_sim         3.600000  3.400000  3.600000   
                    hyper_hypo          4.000000  3.333333  4.333333   

                                            id_3      id_4      id_5  \
question_type word  method                                             
design        chair AutoExtend          4.900000  4.500000  4.400000   
                    Word2vec            3.600000  3.700000  2.000000   
                    analogy_hyper_hypo  4.600000  3.600000  4.000000   
                    analogy_sim         3.800000  4.000000  2.200000   
                    hyper_hypo          4.833333  4.166667  3.833333   
              suit  AutoExtend          4.700000  4.200000  4.400000   
                    Word2vec            3.400000  2.900000  2.000000   
                    analogy_hyper_hypo  4.600000  3.600000  3.800000   
                    analogy_sim         3.200000  2.600000  1.200000   
                    hyper_hypo          4.500000  4.166667  4.333333   
              table AutoExtend          4.400000  3.700000  3.700000   
                    Word2vec            3.800000  4.000000  3.200000   
                    analogy_hyper_hypo  4.200000  3.600000  4.200000   
                    analogy_sim         3.800000  3.600000  2.800000   
                    hyper_hypo          4.833333  4.333333  4.166667   
relation      chair AutoExtend          5.000000  4.400000  4.800000   
                    Word2vec            3.600000  4.300000  2.200000   
                    analogy_hyper_hypo  4.400000  2.800000  3.800000   
                    analogy_sim         3.800000  4.400000  2.600000   
                    hyper_hypo          4.833333  4.166667  4.500000   
              suit  AutoExtend          4.700000  4.100000  4.600000   
                    Word2vec            3.400000  2.900000  1.900000   
           

In [134]:
df_rawdata.groupby(['question_type', 'method']).mean()

id_0      id_1      id_2      id_3  \
question_type method                                                       
design        AutoExtend          3.366667  2.966667  4.133333  4.666667   
              Word2vec            3.033333  3.166667  2.433333  3.600000   
              analogy_hyper_hypo  3.266667  3.466667  3.933333  4.466667   
              analogy_sim         3.133333  3.200000  2.600000  3.600000   
              hyper_hypo          3.666667  3.333333  4.388889  4.722222   
relation      AutoExtend          3.466667  3.233333  4.500000  4.666667   
              Word2vec            3.133333  3.233333  2.700000  3.600000   
              analogy_hyper_hypo  3.333333  3.133333  4.066667  4.266667   
              analogy_sim         3.133333  3.266667  2.600000  3.733333   
              hyper_hypo          3.555556  3.277778  4.500000  4.722222   

                                      id_4      id_5      id_6      id_7  \
question_type method                                                       
design        AutoExtend          4.133333  4.166667  3.933333  4.433333   
              Word2vec            3.533333  2.400000  2.166667  3.266667   
              analogy_hyper_hypo  3.600000  4.000000  3.066667  4.133333   
              analogy_sim         3.400000  2.066667  2.066667  3.066667   
              hyper_hypo          4.222222  4.111111  4.222222  4.444444   
relation      AutoExtend          4.066667  4.533333  4.033333  4.233333   
              Word2vec            3.633333  2.433333  2.500000  3.166667   
              analogy_hyper_hypo  3.466667  4.200000  3.266667  4.066667   
              analogy_sim         3.533333  2.200000  2.266667  2.933333   
              hyper_hypo          4.166667  4.666667  4.444444  4.277778   

                                      id_8      id_9  ...     id_45     id_46  \
question_type method                                  ...                       
design        AutoExtend          4.133333  4.066667  ...  4.500000  3.733333   
              Word2vec            2.633333  3.133333  ...  2.833333  2.033333   
              analogy_hyper_hypo  3.733333  3.666667  ...  4.133333  3.066667   
              analogy_sim         2.533333  2.866667  ...  2.800000  1.933333   
              hyper_hypo          4.000000  4.222222  ...  4.666667  4.166667   
relation      AutoExtend          4.200000  4.233333  ...  4.666667  3.900000   
              Word2vec            2.700000  3.233333  ...  2.866667  2.166667   
              analogy_hyper_hypo  3.800000  4.000000  ...  4.266667  2.866667   
              analogy_sim         2.600000  3.133333  ...  2.466667  2.000000   
              hyper_hypo          4.277778  4.388889  ...  4.500000  3.944444   

                                     id_47     id_48     id_49     id_50  \
question_type method                                                       
design        AutoExtend          4.366667  4.433333  2.800000  3.100000   
              Word2vec            2.900000  2.833333  2.833333  2.933333   
              analogy_hyper_hypo  4.066667  4.066667  2.733333  2.933333   
              analogy_sim         2.666667  2.800000  3.200000  2.933333   
              hyper_hypo          4.333333  4.555556  2.611111  3.166667   
relation      AutoExtend          4.433333  4.500000  2.900000  3.033333   
              Word2vec            3.233333  2.966667  2.800000  3.100000   
              analogy_hyper_hypo  3.533333  4.000000  2.600000  3.133333   
              analogy_sim         2.866667  2.733333  2.866667  3.133333   
              hyper_hypo          4.500000  4.611111  3.000000  2.888889   

                                     id_51     id_52     id_53     id_54  
question_type method                                                      
design        AutoExtend          3.933333  4.500000  4.766667  3.733333  
              Word2vec            2.266667  2.233333  3.500000  2.733333  
              analogy_hype

In [135]:
df_relation['mean_per_question'].plot()

KeyError: 'mean_per_question'

In [ ]:
df_rawdata.groupby('word').mean()

,mean_per_question,id_0,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,...,id_45,id_46,id_47,id_48,id_49,id_50,id_51,id_52,id_53,id_54
word,,,,,,,,,,,,,,,,,,,,,
chair,3.557323,3.736111,3.180556,3.652778,4.333333,4.069444,3.430556,3.611111,3.847222,3.847222,...,3.666667,3.027778,3.736111,4.194444,2.847222,3.000000,3.263889,3.638889,4.291667,3.486111
suit,3.168939,2.597222,3.222222,3.319444,4.069444,3.486111,3.250000,2.847222,3.500000,2.916667,...,3.500000,3.194444,3.277778,3.263889,2.652778,2.986111,2.888889,3.319444,3.944444,3.027778
table,3.511869,3.569444,3.222222,3.736111,4.194444,3.861111,3.750000,3.194444,4.083333,3.638889,...,4.166667,2.791667,4.152778,3.805556,3.000000,3.125000,3.236111,3.333333,4.430556,3.444444


In [ ]:
df_layout

,カラム,アイテム名,回答タイプ,カテゴリ数,質問文A,質問文B,1,2,3,4,...,38,39,40,41,42,43,44,45,46,47
0,1,SAMPLEID,N,NaN,NaN,回答者ID,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ANSWERDATE,D,NaN,NaN,回答日時,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,F1_SEX,SA,2.0,NaN,性別,男性,女性,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,F2_AGEID,SA,12.0,NaN,年齢,〜19歳,20〜24歳,25〜29歳,30〜34歳,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,F3_PREFECTURE,SA,47.0,NaN,都道府県,北海道,青森県,岩手県,宮城県,...,愛媛県,高知県,福岡県,佐賀県,長崎県,熊本県,大分県,宮崎県,鹿児島県,沖縄県
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,229,Q8S32,SA,5.0,以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか,bookshelf(本棚),table(机)に関するアイデアを生み出す際に，全く有効であると思わない,あまり有効であると思わない,どちらともいえない,やや有効であると思う,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,230,Q8S33,SA,5.0,以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか,dresser(ドレッサー),table(机)に関するアイデアを生み出す際に，全く有効であると思わない,あまり有効であると思わない,どちらともいえない,やや有効であると思う,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,231,Q8S34,SA,5.0,以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか,furniture(家具),table(机)に関するアイデアを生み出す際に，全く有効であると思わない,あまり有効であると思わない,どちらともいえない,やや有効であると思う,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,232,Q8S35,SA,5.0,以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか,turntable(ターンテーブル),table(机)に関するアイデアを生み出す際に，全く有効であると思わない,あまり有効であると思わない,どちらともいえない,やや有効であると思う,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_labeldata

,回答者ID,回答日時,性別,年齢,都道府県,回答グループ,回答グループの名称,Q1S1 [例題] 以下の単語は car(車) という単語に関係していると思いますか． plain hot water（白湯）,Q1S2 [例題] 以下の単語は car(車) という単語に関係していると思いますか． doll（人形）,Q1S3 [例題] 以下の単語は car(車) という単語に関係していると思いますか． park（公園）,...,Q8S27 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか linen napkins(亜麻製のナプキン),Q8S28 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか dinning room(ダイニングルーム),Q8S29 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか bookcase(本棚),Q8S30 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか medicine chiropractic podiatry(医学カイロプラクティック足治療),Q8S31 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか tables(テーブル),Q8S32 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか bookshelf(本棚),Q8S33 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか dresser(ドレッサー),Q8S34 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか furniture(家具),Q8S35 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか turntable(ターンテーブル),Q8S36 以下の単語は table(机) に関するアイデアを生み出す際に有効であると思いますか drawers(引き出し)
0,102268301,2022/01/18 14:16:36,1,50〜54歳,兵庫県,2,回答グループ2,car(車)に全く関係していない,やや関係している,やや関係している,...,どちらともいえない,どちらともいえない,やや有効であると思う,どちらともいえない,やや有効であると思う,どちらともいえない,あまり有効であると思わない,どちらともいえない,どちらともいえない,やや有効であると思う
1,103454427,2022/01/18 14:17:51,1,30〜34歳,愛知県,2,回答グループ2,やや関係している,どちらでもない,あまり関係していない,...,どちらともいえない,あまり有効であると思わない,やや有効であると思う,どちらともいえない,どちらともいえない,どちらともいえない,やや有効であると思う,どちらともいえない,どちらともいえない,やや有効であると思う
2,100654697,2022/01/18 14:20:24,1,45〜49歳,京都府,2,回答グループ2,car(車)に全く関係していない,あまり関係していない,あまり関係していない,...,どちらともいえない,やや有効であると思う,やや有効であると思う,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,有効であると思う,やや有効であると思う,どちらともいえない,やや有効であると思う,有効であると思う,どちらともいえない
3,103534250,2022/01/18 14:28:33,1,40〜44歳,神奈川県,2,回答グループ2,あまり関係していない,あまり関係していない,あまり関係していない,...,どちらともいえない,やや有効であると思う,やや有効であると思う,どちらともいえない,有効であると思う,やや有効であると思う,やや有効であると思う,有効であると思う,有効であると思う,どちらともいえない
4,102876990,2022/01/18 14:28:50,1,65〜69歳,神奈川県,2,回答グループ2,car(車)に全く関係していない,car(車)に全く関係していない,car(車)に全く関係していない,...,どちらともいえない,やや有効であると思う,どちらともいえない,どちらともいえない,有効であると思う,どちらともいえない,どちらともいえない,やや有効であると思う,有効であると思う,どちらともいえない
5,101345292,2022/01/18 14:30:18,1,60〜64歳,兵庫県,2,回答グループ2,car(車)に全く関係していない,car(車)に全く関係していない,あまり関係していない,...,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,やや有効であると思う,やや有効であると思う,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,やや有効であると思う,やや有効であると思う,やや有効であると思う,やや有効であると思う,やや有効であると思う,有効であると思う
6,103106982,2022/01/18 14:32:03,1,70歳〜,千葉県,2,回答グループ2,car(車)に全く関係していない,car(車)に全く関係していない,car(車)に全く関係していない,...,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,あまり有効であると思わない,どちらともいえない,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,有効であると思う,どちらともいえない,あまり有効であると思わない,どちらともいえない,どちらともいえない,どちらともいえない
7,103543992,2022/01/18 14:33:59,1,〜19歳,北海道,2,回答グループ2,どちらでもない,どちらでもない,どちらでもない,...,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,有効であると思う,有効であると思う,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,やや有効であると思う,有効であると思う,やや有効であると思う,有効であると思う,有効であると思う,有効であると思う
8,101516785,2022/01/18 14:36:02,1,40〜44歳,京都府,2,回答グループ2,car(車)に全く関係していない,car(車)に全く関係していない,あまり関係していない,...,あまり有効であると思わない,あまり有効であると思わない,やや有効であると思う,table(机)に関するアイデアを生み出す際に，全く有効であると思わない,有効であると思う,やや有効であると思う,やや有効であると思う,やや有効であると思う,有効であると思う,やや有効であると思う
9,102785122,2022/01/18 14:47:33,1,55〜59歳,徳島県,2,回答グループ2,car(車)に全く関係していない,car(車)に全く関係していない,car(車)に全く関係していない,...,あまり有効であると思わない,有効であると思う,やや有効であると思う,どちらともいえない,有効であると思う,やや有効であると思う,あまり有効であると思わない,やや有効であると思う,有効であると思う,有効であると思う
